## Setting up a Hydration Free Energy Calculuation
This notebook will guide you through how to use FESetup in order to produce the inputs needed for an alchemical free energy calculation.    

The notebook forms part of the CCPBio-Sim training.

*Author (2018): Antonia Mey   
Email: antonia.mey@ed.ac.uk*

*Edited (2019): Sarah Fegan*

### Setting up the notebook with all necessary imports

In [ ]:
import nglview as nv

### Setting the stage

We want to compute the relative free energy of hydration between ethane and methanol. That is the free energy difference between hydrating an ethane molecule in water and a methanol molecule in water.

Below you can see a thermodynamic cycle for the relative hydration free energy of ethane and methanol:
![therm_cycle](images/Therm_cycle.png)

We use FESetup to automatically generate the perturbation information needed for running the alchemical free energy calculations. The FESetup documentation can be found at: https://fesetup.readthedocs.io 

### How to use this notebook

FESetup is a predominantly command line based tool. Executing commands in a notebook cell is possible but it is easier to work in a terminal. To open the terminal from the Jupyter home page click on New and then Terminal. To work with this notebook you will have to switch between the terminal tab and the notebook tab. It might be worthwhile separating them out into two windows and arranging them across the screen in such a way that you can look at both the terminal and the notebook. You will be required to edit some files on the command line. This can be done in `nano`, if the notebooks are run as part of the CCPBioSim online training. If you happen to be running this notebook on your computer at home feel free to use whatever editor that works for you to edit required files. 

Lines which begin with $ will be commands that should be entered into the terminal

### Visualise a molecule

Have a look in the `data/01_ethane-methanol` directory.

$ cd data/01_ethane-methanol

$ ls

You will find a file called `morph.in` and a directory called `solutes`. `morph.in` is the main input file and we will look at it more closely in a second. `solutes` contains two further directories: `ethane` and `methanol` each containing a pdb structure file called `solute.pdb` containing the 3D structure of the two molecules we want to compute the relative hydration free energy difference.   

Let's use nglview to look at the ethane molecule:

In [ ]:
view = nv.show_file('data/01_ethane-methanol/solutes/ethane/solute.pdb')
view

**1. Task: load methanol into nglview as well**

In [ ]:
#Insert code for visualizing methanol here



## FESetup
Now, rather than just looking at the structure of these two molecules we want to set them up with FESetup such that they will be ready for running an alchemical free energy simulation. You can open a terminal and run FESetup from the command line by simply typing:   
    ```FESetup```
    
FESetup is a command line tool and you can pass it an input file defining what you would like to set up. If you run FESetup without providing it with an input file, it will list all possible config file options you could use.

$ FESetup 

We will be working with ethane and methanol. Let's have a closer look at the input file `morph.in` provided.

$ nano morph.in

The first section consists of global directives. Important here are the two directives `AFE.type=gromacs` and `AFE.separate_vdw_elec = false`. The first one means that GROMACS compatible output is generated and the second one means that the simulation will be carried out using a single step perturbation, perturbing charges and van der Waals interactions at the same time. Later on we will also look at how to generate output for different simulation software.

For FESetup to understand what files to work on, a strict directory structure is used. `basedir` gives information where it can find all molecules files. Here the basedir is called `solutes`. In `solutes` you have the molecules ethane and methanol as indicated by the corresponding directory and the actual filename which should be the same for all solutes and is in this case `file.name = solute.pdb`. `molecules` defines the molecules that should actually be setup and just lists the directories where we have put the `solutes.pdb` files.

Any directive that starts with `box.type` indicates that a solvated version of the molecule should be generated. FESetup also allows to run a minimisation straight away by using the different `min` directives. In the above example a minimisation for 100 steps using a restraint on non-solvent molecules of 10 will be run. 

To execute FESetup 

$ FESetup morph.in

Let's have a look at the directory again.

$ ls

Some additional files have been generated!

Directories with an underscore `_` are output directories that contain the relevant information for the simulation input. The most useful file is the generated logfile `eth-meth.log`. It contains all commands that were used in order to actually generate the output.  It also contains information whether a setup has failed or not. If you are familiar with AMBER setup tools you will find a lot of familiar looking commands. If you want you can use `nano` on the commandline in order to take a closer look. 

### Solvated molecules
The generated directory `_ligands` contains the original pdb files in a box of water, found in `_ligands/ethane` and `_ligands/methanol`. Let's have a more detailed look at one of the directories. 

$ ls _ligands/methanol/

If you wanted to run a simple MD simulation of methanol in water you can use the `min0001.gro` (coordinate file) and `solvated.top` (topology file) files. The coordinate file contains the energy minimised coordinates as generated by the minimisation protocol in the `morph.in` file. But since we are interested in running an alchemical free energy perturbation from ethane to methanol `_perturbations` is actually the directory we are interested in. 

**Task visualise the solvated molecule `_ligands/methanol/solvated.pdb` using nglview below**

### Perturbation files
Let's take a closer look at `_perturbations`. It has a subdirectory called `gromacs`, because we chose GROMACS to be the alchemical free energy simulation engine and that again has a subdirectory called `ethane~methanol` indicating the perturbation we want to simulate. NOTE: A tilde sign `~` is used rather than a dash, to indicate a perturbation and is standard nomenclature generated by FESetup. 

$ ls _perturbations/gromacs/ethane~methanol/

The `morph.gro` (coordinates) and `morph.top` (topology) files are for the vacuum system and the equivalent files for the solvated system are in the `solvated` directory.

The `pert.itp` file contains the topology for the system. The perturbation information is included in the \[atoms\] section near the top. This section has 11 columns. The atom number, residue number, residue name, atom label, and charge group are the same for both end states. Three things change for morphing one molecule to another: atom type, charge, and mass. The values for the first molecule are listed first and the values for the second molecule are listed in the last three columns. Look at the `pert.itp` file and see if you can identify which atoms are being perturbed.

$ nano _perturbations/gromacs/ethane~methanol/pert.itp

The `pert.atp` file contains the definitions of the atom types with masses and Lennard-Jones parameters.

When you want to use a different software program, the input files will have a different format, but the same infomation about which atoms are changing and the starting and final states will be included.

If you had used `AFE.type=Sire` instead of `AFE.type=gromacs`, then you would have slightly different files in the `_perturbations` directory.

The files of interest would be the `MORPH.onestep.pert` file, `vacuum.parm7` (topology), `vacuum.rst7` (coordinates) and in the directory `solvated`, the solution equivalent `solvated.parm7` and `solvated.rst7` files. The solution and vacuum files will be used to simulate the vacuum and solution leg of the thermodynamic cycle. Let's take a closer look at `MORPH.onestep.pert`

This `MORPH.onestep.pert` file is in the format read by the Sire program. It defines the initial and final atom types and their initial and final Lennard Jones parameters as well as partial charges. The first 2 lines are a header, the rest of the file describes the perturbed atoms. Each perturbed atom has a section that starts with a line `atom` and ends with a line `endatom`. In the perturbation between ethane and methanol the first C atom (C1), will stay a C atom but the partial charges on the molecule will change because of the changed environment from initially having a methyl group attached to it to an OH group attached after the transformation. The second C atom will change to being an oxygen atom after the perturbation. You can now see how the initial atom type is `c3`, and the final atom type is `oh`, meaning that we are going from an sp3 hybridised carbon atom to an oxygen atom that is bonded to a hydrogen. The Lennard-Jones terms change a little and the partial charges do too. The alchemical free energy software can then create a linear perturbation between the initial and final values to slowly morph from one to the other.  

### Adding more perturbations
Now we understand the output generated by FESetup lets see how we can change the input to get a different output. 

**Task: Can you modify the `morph.in` file in such a way that we not only generate a ethane to methanol, but also a methanol to ethane perturbation file**

Using nano open morph.in and identify what you need to change in order to not only generate ethane > methanol, but also methanol > ethane perturbation. Hint: Lists of perturbations are separated by a comma. 

Now run FESetup with the modified input file. 

We should now have an additional perturbation in the `_perturbations/gromacs` directory. 

$ ls _perturbations/gromacs

**Task: visulatise the `vacuum.pdb` file using nglview of the new methanol~ethane perturbation**

In [ ]:
## add code to load vacuum.pdb into nglview



Questions:
- Why does the molecule look so weird, have we done something wrong?
- Can you identify the atom types that are connected to oxygen atom? Think of what was covered in the lecture. 

### Equilibration in FESetup
FESetup can also be used to straightaway equilibrate the solvation box with the solute in it. It may be desirable to run an equilibration protocol with FESetup, this can be done by adding equilibration information to the input file. Have a look at the `equilibrate.in` input file. 

$ nano equilibrate.in

The file now contains a set of directives that will run a NVT simulation to heat the system to 300 K followed by an NPT simulation to equilibrate the density and then a protocol that will slowly release the restraint on the solute molecule in the NPT ensemble. If you want to save the files generated by the previous input (`morph.in`), you need to rename the `_ligands` and `_perturbations` directories before rerunning FESetup.

Now try running with the equilibration steps.

**Do you get an error?** Can you trouble shoot what it might be?

Make sure that the perturbations were actually successfully generated. 

$ ls _perturbations/gromacs

### Running simulations

What do we need to run a free energy calculation? The details of running free-energy simulations depend on the MD software you choose (this workshop is focusing on creating the input files, we will skip the actual running of MD trajectories). In general you need:

- coordinates (included in the FESetup output)
- topology and for some MD programs a separate morph file (included in the FESetup output)
- instructions for the MD program, i.e. config file, mdp file, or control file (you provide this)

Then these inputs are used to run MD simulations at several lambda values. Lambda is a parameter which morphs between the two end states, with values from 0 (molecule A) to 1 (molecule B).